学习R语言标准化技术。

In [1]:
library(tidyverse)

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.4     v readr     2.1.5
v forcats   1.0.0     v stringr   1.5.1
v ggplot2   3.5.1     v tibble    3.2.1
v lubridate 1.9.4     v tidyr     1.3.1
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [5]:
library(broom)

In [2]:
nhefs <- read_csv("./data/nhefs.csv")

head(nhefs)

Rows: 1629 Columns: 64
-- Column specification --------------------------------------------------------
Delimiter: ","
dbl (64): seqn, qsmk, death, yrdth, modth, dadth, sbp, dbp, sex, age, race, ...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


seqn,qsmk,death,yrdth,modth,dadth,sbp,dbp,sex,age,...,birthcontrol,pregnancies,cholesterol,hightax82,price71,price82,tax71,tax82,price71_82,tax71_82
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
233,0,0,NA,NA,NA,175,96,0,42,...,2,NA,197,0,2.183594,1.739990,1.1022949,0.4619751,0.44378662,0.6403809
235,0,0,NA,NA,NA,123,80,0,36,...,2,NA,301,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688
244,0,0,NA,NA,NA,115,75,1,56,...,0,2,157,0,1.569580,1.513428,0.5512695,0.2309875,0.05619812,0.3202515
245,0,1,85,2,14,148,78,0,68,...,2,NA,174,0,1.506592,1.451904,0.5249023,0.2199707,0.05479431,0.3049927
252,0,0,NA,NA,NA,118,77,0,40,...,2,NA,216,0,2.346680,1.797363,1.3649902,0.5718994,0.54931641,0.7929688
257,0,0,NA,NA,NA,141,83,1,43,...,0,1,212,1,2.209961,2.025879,1.1547852,0.7479248,0.18408203,0.4069824


In [3]:
nhefs$cens <- ifelse(is.na(nhefs$wt82), 1, 0)

In [6]:
fit <- glm(
  wt82_71 ~ qsmk + sex + race + age + I(age * age) + as.factor(education)
  + smokeintensity + I(smokeintensity * smokeintensity) + smokeyrs
  + I(smokeyrs*smokeyrs) + as.factor(exercise) + as.factor(active)
  + wt71 + I(wt71*wt71) + qsmk*smokeintensity,
  data=nhefs
)

tidy(fit)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-1.5881656617,4.3130359236,-0.3682245,0.7127562582
qsmk,2.5595940926,0.8091486134,3.1633177,0.0015901758
sex,-1.4302716586,0.4689575529,-3.0498958,0.0023281370
race,0.5601096041,0.5818887784,0.9625716,0.3359131882
age,0.3596352618,0.1633187643,2.2020450,0.0278094360
I(age * age),-0.0061009554,0.0017261355,-3.5344591,0.0004206537
as.factor(education)2,0.7904440282,0.6070004615,1.3022132,0.1930375994
as.factor(education)3,0.5563124031,0.5561015705,1.0003791,0.3172837172
as.factor(education)4,1.4915695237,0.8322703739,1.7921694,0.0733014342


In [7]:
nhefs$predicted.meanY <- predict(fit, nhefs)

In [ ]:
nhefs[which(nhefs$seqn==24770), c("predicted.meanY", "qsmk", "sex",
                                 "race", "age", "education", 
                                 "smokeintensity", "smokeyrs", "exercise",
                                 "active", "wt71")]

predicted.meanY,qsmk,sex,race,age,education,smokeintensity,smokeyrs,exercise,active,wt71
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.3421569,0,0,0,26,4,15,12,1,0,111.58


In [ ]:
summary(nhefs$predicted.meanY[nhefs$cens == 0])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-10.876   1.116   3.042   2.638   4.511   9.876 

In [ ]:
summary(nhefs$wt82_71[nhefs$cens == 0])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
-41.280  -1.478   2.604   2.638   6.690  48.538 

In [11]:
# 合成数据集
id <- c("Rheia", "Kronos", "Demeter", "Hades", "Hestia", "Poseidon", 
  "Hera", "Zeus", "Artemis", "Apollo", "Leto", "Ares", "Athena", 
  "Hephaestus", "Aphrodite", "Cyclope", "Persephone", "Hermes", 
  "Hebe", "Dionysus")
N <- length(id)
L <- c(0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1)
A <- c(0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1)
Y <- c(0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0)
interv <- rep(-1, N)
observed <- cbind(L, A, Y, interv)
untreated <- cbind(L, rep(0, N), rep(NA, N), rep(0, N))
treated <- cbind(L, rep(1, N), rep(NA, N), rep(1, N))
table22 <- as.data.frame(rbind(observed, untreated, treated))
table22$id <- rep(id, 3)

table22

L,A,Y,interv,id
<dbl>,<dbl>,<dbl>,<dbl>,<chr>
0,0,0,-1,Rheia
0,0,1,-1,Kronos
0,0,0,-1,Demeter
0,0,0,-1,Hades
0,1,0,-1,Hestia
0,1,0,-1,Poseidon
0,1,0,-1,Hera
0,1,1,-1,Zeus
1,0,1,-1,Artemis


In [12]:
glm.obj <- glm(Y~ A*L, data = table22)
summary(glm.obj)
table22$predicted.meanY <- predict(glm.obj, table22)

mean(table22$predicted.meanY[table22$interv == -1])
mean(table22$predicted.meanY[table22$interv == 0])
mean(table22$predicted.meanY[table22$interv == 1])


Call:
glm(formula = Y ~ A * L, data = table22)

Coefficients:
              Estimate Std. Error t value Pr(>|t|)
(Intercept)  2.500e-01  2.552e-01   0.980    0.342
A            3.957e-17  3.608e-01   0.000    1.000
L            4.167e-01  3.898e-01   1.069    0.301
A:L         -1.313e-16  4.959e-01   0.000    1.000

(Dispersion parameter for gaussian family taken to be 0.2604167)

    Null deviance: 5.0000  on 19  degrees of freedom
Residual deviance: 4.1667  on 16  degrees of freedom
  (40 observations deleted due to missingness)
AIC: 35.385

Number of Fisher Scoring iterations: 2


[1] 0.5

[1] 0.5

[1] 0.5

In [15]:
library(boot)

# function to calculate difference in means
standardization <- function(data, indices) {
  # create a dataset with 3 copies of each subject
  d <- data[indices, ] # 1st copy: equal to original one`
  d$interv <- -1
  d0 <- d # 2nd copy: treatment set to 0, outcome to missing
  d0$interv <- 0
  d0$qsmk <- 0
  d0$wt82_71 <- NA
  d1 <- d # 3rd copy: treatment set to 1, outcome to missing
  d1$interv <- 1
  d1$qsmk <- 1
  d1$wt82_71 <- NA
  d.onesample <- rbind(d, d0, d1) # combining datasets

  # linear model to estimate mean outcome conditional on treatment and confounders
  # parameters are estimated using original observations only (interv= -1)
  # parameter estimates are used to predict mean outcome for observations with set 
  # treatment (interv=0 and interv=1)
  fit <- glm(
    wt82_71 ~ qsmk + sex + race + age + I(age * age) + 
    as.factor(education) + smokeintensity + 
    I(smokeintensity * smokeintensity) + smokeyrs + I(smokeyrs * smokeyrs) +
    as.factor(exercise) + as.factor(active) + wt71 + I(wt71 * wt71),
    data = d.onesample)

  d.onesample$predicted_meanY <- predict(fit, d.onesample)

  # estimate mean outcome in each of the groups interv=-1, interv=0, and interv=1
  return(
    c(
      mean(d.onesample$predicted_meanY[d.onesample$interv == -1]),
      mean(d.onesample$predicted_meanY[d.onesample$interv == 0]),
      mean(d.onesample$predicted_meanY[d.onesample$interv == 1]),
      mean(d.onesample$predicted_meanY[d.onesample$interv == 1]),
      mean(d.onesample$predicted_meanY[d.onesample$interv == 0])
    )
  )
}

# bootstrap
results <- boot(data = nhefs, statistic = standardization, R = 5)

# generating confidence intervals
se <- c(sd(results$t[, 1]), sd(results$t[, 2]), 
        sd(results$t[, 3]), sd(results$t[, 4]))
mean <- results$t0
ll <- mean - qnorm(0.975) * se
ul <- mean + qnorm(0.975) * se

bootstrap <- data.frame(
  cbind(
    c(
      "Observed", "No Treatment", "Treatment",
      "Treatment - No Treatment"),
    mean,
    se,
    ll,
    ul
  )
)

bootstrap

Warning message in mean - qnorm(0.975) * se:
"longer object length is not a multiple of shorter object length"
Warning message in mean + qnorm(0.975) * se:
"longer object length is not a multiple of shorter object length"
Warning message in cbind(c("Observed", "No Treatment", "Treatment", "Treatment - No Treatment"), :
"number of rows of result is not a multiple of vector length (arg 1)"


V1,mean,se,ll,ul
<chr>,<chr>,<chr>,<chr>,<chr>
Observed,2.56188497106098,0.20410057067334,2.16185520331716,2.9619147388048
No Treatment,1.65212306626743,0.124326594750198,1.40844741823654,1.89579871429833
Treatment,5.11474489549335,0.499728469534772,4.13529509315588,6.09419469783082
Treatment - No Treatment,5.11474489549335,0.499728469534772,4.13529509315588,6.09419469783082
Observed,1.65212306626743,0.20410057067334,1.25209329852361,2.05215283401125
